In [2]:
import numpy as np
import SimpleITK as sitk
import os
import json
from PIL import Image

In [3]:
base_dir = r"/mnt/labnas/DataCollection/Raw/SPECT/cardiac/cardiac_SPECT_raw/patients"
dst_dir = r"/mnt/labnas/DataCollection/Project/RadiSmart/cardiac/all_nii"

fingerprint = {}
cnt = 0
for dirname in sorted(os.listdir(base_dir)):
    if not os.path.isdir(os.path.join(base_dir, dirname)):
        continue
    
    case_info = {
        "pid": dirname
    }
    case_name = f"Patient_{str(cnt).zfill(4)}"
    for filename in os.listdir(os.path.join(base_dir, dirname)):
        dcm_filepath = os.path.join(base_dir, dirname, filename)
        
        try:
            sitk_img = sitk.ReadImage(dcm_filepath)
        except:
            pass
        
        # 截图
        try:
            title = sitk_img.GetMetaData('0008|103e')
            
            # 这个空格很重要
            if title == "ScreenCap ":
                out_dir = os.path.join(dst_dir, case_name)
                if not os.path.exists(out_dir):
                    os.makedirs(out_dir)
                    
                # 保存为图片
                report_img = sitk.GetArrayFromImage(sitk_img)[0].astype(np.uint8)
                image = Image.fromarray(report_img)
                out_filepath = os.path.join(dst_dir, case_name, f"screenCap_{filename}.png")
                image.save(out_filepath)
                
                # 这里是因为有可能有多张 png的情况
                if "ScreenCap" not in case_info:
                    case_info["ScreenCap"] = [out_filepath]
                else:
                    case_info["ScreenCap"].append(out_filepath)
        except Exception as e:
            print(e)
        
        # SA image
        try:
            type_name = sitk_img.GetMetaData('0054|0400')
            if type_name == "STRESS_FBP_SA ":
                out_dir = os.path.join(dst_dir, case_name)
                if not os.path.exists(out_dir):
                    os.makedirs(out_dir)
                    
                out_filepath = os.path.join(dst_dir, case_name, f"SA_{filename}.nii.gz")
                sitk.WriteImage(sitk_img, out_filepath)
                
                # 这里是因为有可能有多张 png的情况
                if "SA_img" not in case_info:
                    case_info["SA_img"] = [out_filepath]
                else:
                    case_info["SA_img"].append(out_filepath)
        except Exception as e:
            print(e)
            
        
        
        # bullseye image
        try:
            # title = sitk_img.GetMetaData('0008|103e')
            type_name = sitk_img.GetMetaData('0054|0400')
            # print(filename, title, type_name)
            if type_name == "STRESS_FBP_Polar":
                out_dir = os.path.join(dst_dir, case_name)
                if not os.path.exists(out_dir):
                    os.makedirs(out_dir)
                    
                out_filepath = os.path.join(dst_dir, case_name, f"Polar_{filename}.nii.gz")
                sitk.WriteImage(sitk_img, out_filepath)
                
                # 这里是因为有可能有多张 png的情况
                if "Polar_img" not in case_info:
                    case_info["Polar_img"] = [out_filepath]
                else:
                    case_info["Polar_img"].append(out_filepath)
        except Exception as e:
            print(e)
        # sitk.WriteImage(img, os.path.join(base_dir, dirname, filename))
    
    if len(case_info)>1:
        fingerprint[case_name] = case_info
        cnt += 1
    # break

** ERROR (nifti_image_write_engine): cannot open output file '/mnt/labnas/DataCollection/Project/RadiSmart/cardiac/all_nii/Patient_0000/SA_101.nii.gz'


Exception thrown in SimpleITK ImageFileWriter_Execute: /tmp/SimpleITK-build/ITK/Modules/IO/NIFTI/src/itkNiftiImageIO.cxx:2375:
ITK ERROR: NiftiImageIO(0x1c193b0): ERROR: nifti library failed to write image: /mnt/labnas/DataCollection/Project/RadiSmart/cardiac/all_nii/Patient_0000/SA_101.nii.gz
Exception thrown in SimpleITK Image_GetMetaData: /tmp/SimpleITK-build/ITK/Modules/Core/Common/src/itkMetaDataDictionary.cxx:77:
ITK ERROR: Key '0054|0400' does not exist 
Exception thrown in SimpleITK Image_GetMetaData: /tmp/SimpleITK-build/ITK/Modules/Core/Common/src/itkMetaDataDictionary.cxx:77:
ITK ERROR: Key '0054|0400' does not exist 
Exception thrown in SimpleITK Image_GetMetaData: /tmp/SimpleITK-build/ITK/Modules/Core/Common/src/itkMetaDataDictionary.cxx:77:
ITK ERROR: Key '0008|103e' does not exist 
Exception thrown in SimpleITK Image_GetMetaData: /tmp/SimpleITK-build/ITK/Modules/Core/Common/src/itkMetaDataDictionary.cxx:77:
ITK ERROR: Key '0054|0400' does not exist 
Exception thrown in S

In [6]:
fingerprint

{'Patient_0000': {'pid': '000149118',
  'SA_img': ['/mnt/labnas/DataCollection/Project/RadiSmart/cardiac/all_nii/Patient_0000/SA_112.nii.gz'],
  'ScreenCap': ['/mnt/labnas/DataCollection/Project/RadiSmart/cardiac/all_nii/Patient_0000/screenCap_110.png'],
  'Polar_img': ['/mnt/labnas/DataCollection/Project/RadiSmart/cardiac/all_nii/Patient_0000/Polar_107.nii.gz']},
 'Patient_0001': {'pid': '000352053',
  'SA_img': ['/mnt/labnas/DataCollection/Project/RadiSmart/cardiac/all_nii/Patient_0001/SA_9.nii.gz',
   '/mnt/labnas/DataCollection/Project/RadiSmart/cardiac/all_nii/Patient_0001/SA_20.nii.gz'],
  'ScreenCap': ['/mnt/labnas/DataCollection/Project/RadiSmart/cardiac/all_nii/Patient_0001/screenCap_18.png'],
  'Polar_img': ['/mnt/labnas/DataCollection/Project/RadiSmart/cardiac/all_nii/Patient_0001/Polar_15.nii.gz']},
 'Patient_0002': {'pid': '000652515',
  'SA_img': ['/mnt/labnas/DataCollection/Project/RadiSmart/cardiac/all_nii/Patient_0002/SA_154.nii.gz'],
  'Polar_img': ['/mnt/labnas/DataC

In [8]:
fingerprint_filepath = os.path.join(dst_dir, "fingerprint.json")
with open(fingerprint_filepath, "w") as fout:
    json.dump(fingerprint, fout, indent=4)